In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# --- 1. Prepare Dataset (Example using a small subset of cats vs dogs) ---
# Assuming you have a directory structure like this:
# data/
#   train/
#     cats/
#       cat.0.jpg
#       cat.1.jpg
#       ...
#     dogs/
#       dog.0.jpg
#       dog.1.jpg
#       ...
#   validation/
#     cats/
#       ...
#     dogs/
#       ...

train_dir = 'data/train'  # Replace with your actual train directory
validation_dir = 'data/validation' # Replace with your actual validation directory

if not os.path.exists(train_dir) or not os.path.exists(validation_dir):
    print("Please create 'data/train' and 'data/validation' directories with subdirectories 'cats' and 'dogs'.")
    exit()

# Image preprocessing
img_width, img_height = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # For cats vs dogs
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

# --- 2. Load Pre-trained Model (MobileNetV2) ---
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

# --- 3. Add Custom Classifier (Top Layers) ---
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x) # Binary classification

model = Model(inputs=base_model.input, outputs=predictions)

# --- 4. Compile the Model ---
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# --- 5. Train the Model ---
epochs = 10 # Adjust as needed

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# --- 6. Optional: Fine-tuning (Unfreeze some layers) ---
# Unfreeze some of the top layers of the base model
# unfreeze_at = 100 # Example: Unfreeze layers from the 100th layer onwards.
# for layer in base_model.layers[unfreeze_at:]:
#     layer.trainable = True
#
# model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), # Use a lower learning rate
#               loss='binary_crossentropy', metrics=['accuracy'])
#
# history_fine = model.fit(train_generator,
#                          steps_per_epoch=train_generator.samples // batch_size,
#                          epochs=5, # Fine-tune for fewer epochs
#                          validation_data=validation_generator,
#                          validation_steps=validation_generator.samples // batch_size)

print("Training complete!")

# --- 7. Evaluation (Optional) ---
# model.evaluate(validation_generator)

2025-02-24 16:14:31.853589: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-24 16:14:31.853622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-24 16:14:31.854617: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-24 16:14:31.860128: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Please create 'data/train' and 'data/validation' directories with subdirectories 'cats' and 'dogs'.


FileNotFoundError: [Errno 2] No such file or directory: 'data/train'

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

# 1. Prepare a small dataset (replace with your own)
# For simplicity, we'll generate dummy data. In a real scenario, you'd use actual image data.
# This dummy data simulates two classes: "cats" and "dogs"
def create_dummy_dataset(num_samples=100, image_size=(224, 224), num_classes=2):
    images = np.random.rand(num_samples, image_size[0], image_size[1], 3)
    labels = np.random.randint(0, num_classes, num_samples)
    return images, labels

train_images, train_labels = create_dummy_dataset(num_samples=80)
test_images, test_labels = create_dummy_dataset(num_samples=20)

# 2. Load a pre-trained model (MobileNetV2)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 3. Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# 4. Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)  # 2 classes: cats and dogs

model = Model(inputs=base_model.input, outputs=predictions)

# 5. Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 6. Train the model on the new dataset
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

# 7. Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# 8. Unfreeze some layers for fine-tuning (optional)
# After training the top layers, you can unfreeze some of the convolutional layers
# and train the entire network again with a very low learning rate.
# This step improves accuracy further.

unfreeze_at = 100  # Example: Unfreeze layers starting from the 100th layer
for layer in model.layers[unfreeze_at:]:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Fine tuned Test Loss: {loss}, Fine tuned Test Accuracy: {accuracy}")

#Example using ImageDataGenerator and real file paths.
"""
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'data/train',  # Replace with your training data directory
        target_size=(224, 224),
        batch_size=32,
        class_mode='sparse')

validation_generator = test_datagen.flow_from_directory(
        'data/validation', #Replace with your validation data directory
        target_size=(224, 224),
        batch_size=32,
        class_mode='sparse')

model.fit(train_generator, epochs=10, validation_data=validation_generator)
"""